## Trim raw reads using trimmomatic 
-------


In [ ]:
cd /path/to/raw/reads

for R1 in *R1*
do
   R2=${R1//R1.fastq.gz/R2.fastq.gz}
   R1paired=${R1//.fastq.gz/_paired.fastq.gz}
   R1unpaired=${R1//.fastq.gz/_unpaired.fastq.gz}
   R2paired=${R2//.fastq.gz/_paired.fastq.gz}
   R2unpaired=${R2//.fastq.gz/_unpaired.fastq.gz} \
   java -jar trimmomatic-0.39.jar PE $R1 $R2 $R1paired $R1unpaired $R2paired $R2unpaired \
   ILLUMINACLIP:TruSeq3-PE.fa:2:30:10 LEADING:5 TRAILING:5 SLIDINGWINDOW:4:15 MINLEN:100
done

## RUN FASTQC #TODOOOOOO

## Assembly with SPADES 
-----

In [ ]:
for file1 in /path/to/trimmed/data/directory/*R1*fastq
do 
file2=${file1/R1/R2}
out=${file1%%_R1.fastq}_output
spades.py -k 21,33,55,77 -c --only-assembler -1 $file1 -2 $file2 -o $out
done

## Ref Alignment 
------

In [ ]:
##John  Stuff

## SV Calling 
-----

In [1]:
##local
projectD=/home/sarah/WGS3
cd $projectD
svFolder=testSV

In [ ]:
sftp sharrison@mutation.uncc.edu
#from within sftp
svFolder=testSV
mkdir $svFolder
cd $svFolder
put -r bams
put -r refs
put -r scripts

In [ ]:
##on cluster

svFolder=testSV
cd $svFolder
chmod u+x runScript.sh
mydata=$(pwd)
refs=$mydata/refs
bams=$mydata/bams
./runScript.sh -d $mydata -b $bams -r /home/sharrison/testSV/refs/GCF_003019965.1_ASM301996v1_genomic.fna -i 250

In [ ]:
###local
cd $svFolder
mkdir -p mergeSV
mkdir -p manta
mkdir -p pindel
mkdir -p lumpy
mkdir -p delly
mkdir -p gridss
mkdir -p breakdancer
mkdir -p stats

In [ ]:
sftp sharrison@mutation.uncc.edu
get -r manta
get -r pindel
get -r lumpy
get -r delly
get -r gridss
get -r breakdancer
get -r stats

In [ ]:
##converting delly from bcf to vcf 
cd $projectD/$svFolder/delly
for file in *.bcf ;
do
out=${file%.bcf}.vcf
bcftools view -Ov -o $out $file
done

##annotating gridss output
chmod u+x $projectD/$svFolder/structurial_annotation.R 
cd $projectD/$svFolder/gridss
for file in *.vcf ;
do
outVcf=${file%.vcf}.ann.vcf
outBed=${file%.vcf}.bed
Rscript $projectD/$svFolder/structurial_annotation.R $file $outVcf $outBed
done

cd $projectD/$svFolder/lumpy
for file in *.vcf ;
do
outVcf=${file%.vcf}.ann.vcf
outBed=${file%.vcf}.bed
Rscript $projectD/$svFolder/structurial_annotation.R $file $outVcf $outBed
done

In [4]:
python3

Restarting kernel "Bash"


Keyboard Interrupt


In [2]:
for b in $projectD/$svFolder/stats/*.stats
do
tmp=${b##*/}
sample=${tmp%.sorted.stats}
python3 $projectD/$svFolder/reformat_merge.py $projectD/$svFolder $sample
done

Traceback (most recent call last):
  File "/home/sarah/WGS3/testSV/reformat_merge.py", line 1, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'
Traceback (most recent call last):
  File "/home/sarah/WGS3/testSV/reformat_merge.py", line 1, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'



In [ ]:
numSamples=2
python3 $projectD/$svFolder/overlapSamples.py $projectD/$svFolder/mergeSV $numSamples

In [ ]:
cd $svFolder
mkdir -p svImages 
bedtools igv -slop 10 -path svImages -i $projectD/$svFolder/mergeSV/noShareSV.bed > $projectD/$svFolder/svImages/picScript.sh

In [ ]:
### TODO generate images into IGV make session files 

In [ ]:
cd /path/to/data/mergeSV/finalSV
for photo in *.png ; do convert $photo -rotate -90 $photo ; done

SNP/INDEL calling 

In [ ]:
#John Stuff

Phylogeny